In [1]:
import os

os.chdir("../")
%pwd

'd:\\DHIRAJ\\Data_Science\\Jupyter_Workspace\\Projects\\SpaceX-Falcon-9-first-stage-Landing-Prediction'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    scaler_ckpt: Path
    model_ckpt: Path

In [3]:
from SpaceXF9LandingPred.constants import *
from SpaceXF9LandingPred.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, 
                config_filepath=CONFIG_FILE_PATH, 
                params_filepath=PARAMS_FILE_PATH):
        
        self.config_file=read_yaml(config_filepath)
        self.params_file=read_yaml(params_filepath)

        create_directories([self.config_file.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        
        config=self.config_file.model_trainer
        params=self.params_file.training_params
        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            scaler_ckpt=config.scaler_ckpt,
            model_ckpt=config.model_ckpt
        )

        return model_trainer_config, params
    

In [ ]:
import pandas as pd
import numpy as np

from SpaceXF9LandingPred.logging import logger

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import joblib

class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig, params):
        self.config=config
        self.params=params
        self.scaler=StandardScaler()

    def train(self):
        data = pd.read_csv(self.config.data_path)
        X = data.drop(columns=['Class'], axis=1)
        Y = data['Class'].to_numpy()

        X = self.scaler.fit_transform(X)

        joblib.dump(self.scaler, self.config.scaler_ckpt)

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)
        
        parameters=dict(self.params.svc)
        model=SVC(**parameters)
        model.fit(X_train, Y_train)
        accuracy=model.score(X_test, Y_test)

        logger.info(f"Model trained with accuracy score of {accuracy*100:.2f}% on test data.")

        joblib.dump(model, self.config.model_ckpt)

        logger.info("Model Checkpoint is created in artifacts")

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config, params = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config, params=params)
    model_trainer.train()
except Exception as e:
    raise e

[2025-08-11 16:41:50,194: INFO: common: yaml file : config\config.yaml loaded successfully]
[2025-08-11 16:41:50,209: INFO: common: yaml file : params.yaml loaded successfully]
[2025-08-11 16:41:50,214: INFO: common: created directory at : artifacts]
[2025-08-11 16:41:50,217: INFO: common: created directory at : artifacts/model_trainer]
[2025-08-11 16:41:50,253: INFO: 1934501768: Model trained with accuracy score of 94.12% on test data.]
[2025-08-11 16:41:50,258: INFO: 1934501768: Model Checkpoint is created in artifacts]
